In [58]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("/home/a_parida/MantaFlow/manta/tensorflow/tools/")
import uniio
import glob
import random
import scipy.misc
import os
basePath = 'data/'

In [59]:
trainingEpochs = 10
batchSize      = 10
inSize         = 64 * 64 * 1

In [3]:
vel_files= glob.glob('./data/**/*.uni', recursive=True)
# load data
velocities = []

In [5]:
for uniPath in vel_files:
    header, content = uniio.readUni(uniPath)# returns [Z,Y,X,C] np array
    h = header['dimX']
    w  = header['dimY']
    arr = content[:, ::-1, :, :]  # reverse order of Y axis
    arr = np.reshape(arr, [w, h, 3])
    arr=arr[:,:,1:]# discard Z from [Z,Y,X]
    velocities.append( arr )
    
loadNum = len(velocities)
if loadNum<200:
	print("Error - use at least two full sims, generate data by running 'manta ./manta_genSimSimple.py' a few times..."); exit(1)

velocities = np.reshape( velocities, (len(velocities), 64,64,2) )

print("Read uni files, total data " + format(velocities.shape) )
valiSize = max(100, int(loadNum * 0.1)) # at least 1 full sim...
valiData = velocities[loadNum-valiSize:loadNum,:]
velocities  = velocities[0:loadNum-valiSize,:]
print("Split into %d training and %d validation samples" % (velocities.shape[0], valiData.shape[0]) )
loadNum = velocities.shape[0]

Read uni files, total data (1200, 64, 64, 2)
Split into 1080 training and 120 validation samples


In [6]:
def convolution2d(input, biases, weights, strides, padding_kind='SAME'):
    input = tf.nn.conv2d(input, weights, strides, padding=padding_kind)
    input = tf.nn.bias_add(input, biases)
    input = tf.nn.leaky_relu(input)
    return input

In [35]:
# the network structure 
xIn = tf.placeholder(tf.float32, shape=[None, 64,64, 2])
y = tf.placeholder(tf.float32, shape=[None, 64,64,2])
xOut=tf.reshape(y, shape=[-1, 64*64*2])

In [36]:
weights1=tf.Variable(tf.random_normal([64,64,2,2], stddev=0.01))#weights==filters
#[filter_height, filter_width, in_channels, out_channels]
#bias=out_channels
bias1=tf.Variable(tf.random_normal([2], stddev=0.01))
stride1=[1,4,4,1]
out1=convolution2d(xIn,bias1,weights1,stride1)

In [37]:
weights2=tf.Variable(tf.random_normal([16,16,2,4], stddev=0.01))#weights==filters
#[filter_height, filter_width, in_channels, out_channels]
#bias=out_channels
bias2=tf.Variable(tf.random_normal([4], stddev=0.01))
stride2=[1,4,4,1]
out2=convolution2d(out1,bias2,weights2,stride2)

In [38]:
out2 = tf.reshape(out2, shape=[-1, 64 ]) # flatten
fc_1weights = tf.Variable(tf.random_normal([64, 64*64*2], stddev=0.01))
fc_1bias   = tf.Variable(tf.random_normal([64*64*2], stddev=0.01))
fc1 = tf.add(tf.matmul(out2, fc_1weights), fc_1bias)
fc1 = tf.nn.tanh(fc1)


In [41]:
cost = tf.nn.l2_loss(fc1 - xOut)
opt  = tf.train.AdamOptimizer(0.0001).minimize(cost)


In [42]:
cost

<tf.Tensor 'L2Loss:0' shape=() dtype=float32>

In [43]:
opt

<tf.Operation 'Adam' type=NoOp>

In [44]:
print("Starting training...")
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Starting training...


In [61]:
def velocityFieldToPng(frameArray):
    """ Returns an array that can be saved as png with scipy.misc.toimage
    from a velocityField with shape [height, width, 2]."""
    outputframeArray = np.zeros((frameArray.shape[0], frameArray.shape[1], 3))
    for x in range(frameArray.shape[0]):
        for y in range(frameArray.shape[1]):
            # values above/below 1/-1 will be truncated by scipy
            frameArray[y][x] = (frameArray[y][x] * 0.5) + 0.5
            outputframeArray[y][x][0] = frameArray[y][x][0]
            outputframeArray[y][x][1] = frameArray[y][x][1]
    return outputframeArray

In [60]:
for epoch in range(trainingEpochs):
	batch = []
	for currNo in range(0, batchSize):
		r = random.randint(0, loadNum-1)
		batch.append( velocities[r] )

	_ , currentCost = sess.run([opt, cost], feed_dict={xIn: batch, y: batch})
	if epoch%10==9 or epoch==trainingEpochs-1:
		[valiCost,vout] = sess.run([cost, fc1], feed_dict={xIn: valiData, y: valiData})
		print("Epoch %d/%d: cost %f , validation cost %f " % (epoch, trainingEpochs, currentCost, valiCost) )

		if epoch==trainingEpochs-1:
			outDir = "./test_simple"
			if not os.path.exists(outDir): os.makedirs(outDir)
			print("\n Training done. Writing %d images from validation data to directory %s..." % (len(valiData),outDir) )
			for i in range(len(valiData)):
				scipy.misc.toimage(velocityFieldToPng( np.reshape(valiData[i], [64, 64,2])) , cmin=0.0, cmax=1.0).save("%s/in_%d.png" % (outDir,i))
				scipy.misc.toimage(velocityFieldToPng( np.reshape(vout[i]    , [64, 64,2])) , cmin=0.0, cmax=1.0).save("%s/out_%d.png" % (outDir,i))




Epoch 9/100: cost 24.155813 , validation cost 328.309265 
Epoch 19/100: cost 30.454882 , validation cost 337.375946 
Epoch 29/100: cost 11.694822 , validation cost 334.139130 
Epoch 39/100: cost 26.331802 , validation cost 329.936890 
Epoch 49/100: cost 14.953149 , validation cost 328.907623 
Epoch 59/100: cost 23.145868 , validation cost 329.361023 
Epoch 69/100: cost 44.699745 , validation cost 327.370178 
Epoch 79/100: cost 33.741924 , validation cost 329.561340 
Epoch 89/100: cost 24.089920 , validation cost 325.685791 
Epoch 99/100: cost 31.191666 , validation cost 322.962677 

 Training done. Writing 120 images from validation data to directory data//test_simple...


/home/a_parida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


ValueError: 'arr' does not have a suitable array shape for any mode.

In [46]:
velocities.shape[0]

1080

In [47]:
64*64*2


8192

In [49]:
import random